In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 7.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=a776a471f9eb70d2574c471de03da627a570088eab796bda1ff6cf2353b87c67
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 152 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kdxix712
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kdxix712
     |████████████████████████████████| 132 kB 7.3 MB/s 
     |████████████████████████████████| 4.5 MB 58.1 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 8.9 MB 49.7 MB/s 
     |████████████████████████████████| 127 kB 58.7 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=8f9328be7b2632d1ba0d129a024fa9e54ca9e423697912512b05f5079eebceb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_sfvav0/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3n376ug2/kobert-tokenizer_4f57e83112714f4e8d531fe67bad09c2
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-3n376ug2/kobert-tokenizer_4f57e83112714f4e8d531fe67bad09c2
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=47d382d996d6360ab501345d24b9978ce06f5f080046b90ed65fdf3ae821cf0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-hrnym1l_/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model # 파이토치 용 koBERT모델

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import BertForSequenceClassification, BertConfig

In [7]:
device = torch.device('cuda:0')

In [8]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla T4


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


In [12]:
df = pd.read_csv('data_labelled_yujeong.csv')
df = df[['sentences', 'label']]
df

,sentences,label
0,다른 곳에서 찍은 영상을 보고 교육청 유튜브 컨텐츠 오디션 제의가 들어와 운좋게 합...,3
1,전철은 한가했다. 출근시간 한참 전이라 그런가. 빈자리가 많아서 자리를 하나 차지한...,0
2,지도에 검색해보니 서울역까지 간 후 버스를 타면 8시 53분 도착 예정이길래 서울역...,0
3,그렇게 불편한 도착을 마친 후 간단한 대화가 끝나고 유치원으로 떠났다. 나는 7살사...,8
4,"내 팔다리를 하나씩 잡으며, 나에게 찰싹 달라붙었다.",8
...,...,...
44099,우리가 어디서 어떻게\n만나게 되었고 서로 알아봤는지\n외롭고 힘이 들면 들리는 노...,5
44100,사랑했었던\n설레었던 기억들이\n사랑한다며\n속삭이던 너의 목소리가\n아무 소용 없...,0
44101,참 어렸었지 뭘 몰랐었지\n설레는 젊음 하나로 그땐 그랬지\n참 느렸었지\n늘 지루...,0
44102,살아선 나 안되겠니 너의 사랑을 받는 일\n죽어도 넌 안되겠니 나를 허락하는 일\n...,8


In [13]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1') # 이미 사전학습 모델을 토크나이저로 사용
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False) # BERT 모델도 사전학습된 모델로 사용
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]') # koBERT vocab 으로 토크나이징 하고 빈 부분 PAD로 패딩

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [14]:
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1") .?
# model = AutoModel.from_pretrained("skt/kobert-base-v1") >?

In [15]:
df

,sentences,label
0,다른 곳에서 찍은 영상을 보고 교육청 유튜브 컨텐츠 오디션 제의가 들어와 운좋게 합...,3
1,전철은 한가했다. 출근시간 한참 전이라 그런가. 빈자리가 많아서 자리를 하나 차지한...,0
2,지도에 검색해보니 서울역까지 간 후 버스를 타면 8시 53분 도착 예정이길래 서울역...,0
3,그렇게 불편한 도착을 마친 후 간단한 대화가 끝나고 유치원으로 떠났다. 나는 7살사...,8
4,"내 팔다리를 하나씩 잡으며, 나에게 찰싹 달라붙었다.",8
...,...,...
44099,우리가 어디서 어떻게\n만나게 되었고 서로 알아봤는지\n외롭고 힘이 들면 들리는 노...,5
44100,사랑했었던\n설레었던 기억들이\n사랑한다며\n속삭이던 너의 목소리가\n아무 소용 없...,0
44101,참 어렸었지 뭘 몰랐었지\n설레는 젊음 하나로 그땐 그랬지\n참 느렸었지\n늘 지루...,0
44102,살아선 나 안되겠니 너의 사랑을 받는 일\n죽어도 넌 안되겠니 나를 허락하는 일\n...,8


In [16]:
# 데이터 셋 토크나이징 함수

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44104 entries, 0 to 44103
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentences  44104 non-null  object
 1   label      44104 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 689.2+ KB


In [18]:
# 0, 1, 2, 3, 5, 6, 7, 8 
# 8을 4로 변경 했다
# 상관없겠지?

for i in range(0, len(df['label'])):
  if df['label'][i] == 8:
    df['label'][i] = 4
  else:
    pass

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,sentences,label
0,다른 곳에서 찍은 영상을 보고 교육청 유튜브 컨텐츠 오디션 제의가 들어와 운좋게 합...,3
1,전철은 한가했다. 출근시간 한참 전이라 그런가. 빈자리가 많아서 자리를 하나 차지한...,0
2,지도에 검색해보니 서울역까지 간 후 버스를 타면 8시 53분 도착 예정이길래 서울역...,0
3,그렇게 불편한 도착을 마친 후 간단한 대화가 끝나고 유치원으로 떠났다. 나는 7살사...,4
4,"내 팔다리를 하나씩 잡으며, 나에게 찰싹 달라붙었다.",4
...,...,...
44099,우리가 어디서 어떻게\n만나게 되었고 서로 알아봤는지\n외롭고 힘이 들면 들리는 노...,5
44100,사랑했었던\n설레었던 기억들이\n사랑한다며\n속삭이던 너의 목소리가\n아무 소용 없...,0
44101,참 어렸었지 뭘 몰랐었지\n설레는 젊음 하나로 그땐 그랬지\n참 느렸었지\n늘 지루...,0
44102,살아선 나 안되겠니 너의 사랑을 받는 일\n죽어도 넌 안되겠니 나를 허락하는 일\n...,4


In [19]:
data_list = []
for sentence, label in zip(df['sentences'], df['label']):
  data = []
  data.append(sentence)
  data.append(str(label))

  data_list.append(data)

# sentences 와 label을 새로운 리스트에 저장
# 이부분 왜 하는지 모르겠는데
# 아마 훈련시킬때 리스트를 넣어야 하나보네

In [20]:
data_list

[['다른 곳에서 찍은 영상을 보고 교육청 유튜브 컨텐츠 오디션 제의가 들어와 운좋게 합격했고, 출연자가 되어 유치원에서 촬영을 하게 되었다. 평소 아이들을 좋아하는 나인지라 좋은 기회가 되겠구나 생각해 흔쾌히 승낙을 하였다. 목적지까지 2시간 반 정도 걸리는 곳이기에 다음 날 9시 까지 가려면 늦어도 6시엔 타야 여유있게 도착하지 않을까 싶어. 전날 10시부터 억지로 잠을 잤다. 눈을 뜨니 5시였다. 다행이었다. 졸린 눈을 비비며 양치와 면도를 하고 재빨리 옷을 입은 후 아침 6시에 있는 열차를 타기 위해 역까지 마구 달렸다.',
  '3'],
 ['전철은 한가했다. 출근시간 한참 전이라 그런가. 빈자리가 많아서 자리를 하나 차지한 후 가방 안의 비눗방울을 보고 인사했다. 어떻게든 이따 비눗방울로 놀아줄 생각을 하니 벌써부터 기분이 좋았다.',
  '0'],
 ['지도에 검색해보니 서울역까지 간 후 버스를 타면 8시 53분 도착 예정이길래 서울역에서 버스를 탔다. 하지만 교통 상황 때문인지 갑자기 도착 예정 시간이 9시20분으로 뜨는게 아닌가. 잘못 본건가 싶어 새로고침을 백번도 넘게 하다 기사님께 9시 안에 도착할 것 같냐고 여쭤보았고, 기사님은 고개를 저으셨다. 그렇게 한숨을 푹푹 쉬다 아무 근처 역에 내려 전철시간을 찍어보니 약속 시간보다 도착시간이 5분 늦게 나왔다. 혹시 몰라서 3시간 전에 나왔는데도 지각을 할 위기에 너무 짜증이 났지만, PD님께 정중히 죄송하다고 메시지를 보내던 와중에 학교 담임선생님께 오늘 잘해라잉 메시지가 왔다. 한 분에겐 죄송해야 할 상황인데 한 분에겐 감사하다고 보내야 하는 기분이 꽤 묘했다.',
  '0'],
 ['그렇게 불편한 도착을 마친 후 간단한 대화가 끝나고 유치원으로 떠났다. 나는 7살사랑반친구들의 선생님이 되었는데, 도착 후 간단한 소개를 마치자마자 아이들이 나에게 쏟아졌다.',
  '4'],
 ['내 팔다리를 하나씩 잡으며, 나에게 찰싹 달라붙었다.', '4'],
 ['아이들이 장난감 음식을 이용해 나에게 스

In [21]:
data_list[0]

['다른 곳에서 찍은 영상을 보고 교육청 유튜브 컨텐츠 오디션 제의가 들어와 운좋게 합격했고, 출연자가 되어 유치원에서 촬영을 하게 되었다. 평소 아이들을 좋아하는 나인지라 좋은 기회가 되겠구나 생각해 흔쾌히 승낙을 하였다. 목적지까지 2시간 반 정도 걸리는 곳이기에 다음 날 9시 까지 가려면 늦어도 6시엔 타야 여유있게 도착하지 않을까 싶어. 전날 10시부터 억지로 잠을 잤다. 눈을 뜨니 5시였다. 다행이었다. 졸린 눈을 비비며 양치와 면도를 하고 재빨리 옷을 입은 후 아침 6시에 있는 열차를 타기 위해 역까지 마구 달렸다.',
 '3']

In [22]:
# 하이퍼파라미터 설정
# BERTDataset 이랑 학습 클래스에 넣어줘야한다

max_len = 64
batch_size = 32 # 64로 했더니 터졌다 그래서 32로 했다
warmup_ratio = 0.1
num_epochs = 1 # 코랩 런타임 문제로 인해 너무 오래걸리기도해서 1번
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5 # 이건 자유롭게 조정하며 성능 확인해야할거같다

In [23]:
# 이 부분이 학습데이터 만든는 부분
# BERTDataset([문장, 라벨 리스트], 첫 인덱스, 몇 인덱스씩 넘길지?(예상), 토크나이저, 보캡뭉치, 최대 길이, 패딩, 쌍(?))
# 이부분까지 토크나이징을 하지 않고 보내줘야하기 때문에 유의!

tok = tokenizer.tokenize
train_data = BERTDataset(data_list, 0, 1, tok, vocab, max_len, True, False)

In [24]:
train_data[0]
# BERT 모델은 문장의 시작에 2, 끝에 3 으로 채워준다

(array([   2, 1567, 1021, 6903, 4401, 3380, 7088, 2358,  517, 5492, 3596,
         517, 7514, 7621, 7478, 3421, 4128, 7095, 5330, 1804, 6983, 3513,
        7271, 5400, 4985, 7865,   46, 4579, 7148, 1767, 3594, 7022, 4518,
        7088,  517, 7784,  517, 5896,   54, 4848, 3123, 7088, 4207, 7794,
        1370, 7123, 6003, 4209, 1307, 5330, 1763, 5405, 5496, 2709,  517,
        7986, 7548, 7996, 2949, 5661, 7088, 4924, 6946,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [25]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 5)

# 데이터 로더 , 학습데이터 배치사이즈로 나눠주고 몇명의 노예가 일할지 정한다

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [26]:
# 분류 함수 지정

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes) # 선형
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel, dr_rate = 0.5)

In [28]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [29]:
train_history=[]
loss_history=[]
for e in range(num_epochs):
  train_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long()
    segment_ids = segment_ids.long()
    valid_length= valid_length
    label = label.long()
    out = model(token_ids, valid_length, segment_ids)
         
    #print(label.shape,out.shape)
    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      train_history.append(train_acc / (batch_id+1))
      loss_history.append(loss.data.cpu().numpy())
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1379 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.1883716583251953 train acc 0.0625
epoch 1 batch id 201 loss 1.050472617149353 train acc 0.5694962686567164
epoch 1 batch id 401 loss 0.6410984396934509 train acc 0.6118298004987531
epoch 1 batch id 601 loss 1.1099810600280762 train acc 0.6263519134775375
epoch 1 batch id 801 loss 1.0943858623504639 train acc 0.630461922596754
epoch 1 batch id 1001 loss 0.7934913039207458 train acc 0.6360514485514486
epoch 1 batch id 1201 loss 0.9131069779396057 train acc 0.640429850124896
epoch 1 train acc 0.635038977519942


In [30]:
torch.save(model.state_dict(), 'model.pt')
torch.save(model, 'model_all.pt')

---

# gluonnlp

In [ ]:
from gluonnlp.data import SentencepieceTokenizer
from kobert import get_tokenizer
tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)

In [ ]:
tokenized_data = []

for i in range(0, len(data_list)):
  tokenized_data.append(sp(data_list[i][0]))

tokenized_data